In [1]:
import numpy as np
import scipy.optimize as optimize
import matplotlib.pyplot as plt
from functools import reduce

# Define a function for constructing the system hamiltonian
def construct_h_super(aux_v, aux_e):
    h = np.zeros((n_super,n_super))
    h[:nphys,:nphys] = h_phys
    h[nphys:,nphys:] = np.diag(aux_e)
    h[:nphys,nphys:] = aux_v
    h[nphys:,:nphys] = aux_v.T
    assert(np.allclose(h,h.T))

    return h

# Define a function for constructing X-matrix
def construct_X(e, v, nmom):
    d = np.diag(np.power(e, nmom))
    X = reduce(np.dot, (v[:nphys,:], d, v[:nphys,:].T))
    return X
    
# Define a function for constructing 
def construct_T(nmom):
    if nmom == 0:
       T = np.eye(nphys)
    else:
       T = np.random.random((nphys, nphys))
    return T

# Define a function for calculating the error and reconstructing arrays after error
def mom_err(aux_param, T, grads=False):

   aux_e_opt = aux_param[:naux].copy() 
   aux_v_opt = np.reshape(aux_param[naux:].copy(), (nphys,naux))

   h = construct_h_super(aux_v_opt, aux_e_opt)
   e, v = np.linalg.eigh(h)

   # Construct all the X matrices here
   X = [construct_X(e, v, mom)  for mom in range(n+1)]

   # sum_{n}  w_n | MF_mom(n) - HF_mom(n) |^2
   err = sum([np.sum((X[mom]-T[mom])**2)  for mom in range(n+1)]) 

   if grads:
      print('I cannot do this yet')
      assert(0)
     # deriv = ....
     # grad = 2 * (T-X) * deriv
     # return err, grad
   else:
      return err


#def calc_deriv():
# 
#    ......
#
#return deriv

def fitting(opt_params, grads=False):

    maxiter = 1000
    if grads:
       res = scipy.optimize.minimize(mom_err, opt_params, args=(T,True),
                method='l-bfgs-b',jac=True,tol=1e-9,options={'disp':True,'ftol':1.e-9})
    else:
       res = optimize.minimize(mom_err, opt_params, args=(T),
           method='nelder-mead',tol=1e-9,options={'xtol':1e-8,'disp':True,'maxiter':maxiter,'maxfev':maxiter})

    opt_aux_e = res.x[:naux]
    opt_aux_v = np.reshape(res.x[naux:], (nphys, naux))

    return opt_aux_e, opt_aux_v


# Compute the Physical system eigenvalues and eigenvectors
n = 2
naux = 4
nphys = 1
n_super = nphys + naux
h_phys = np.random.random((nphys,nphys))    # h_alpha,beta
h_phys += h_phys.T
aux_v_init = np.random.random((nphys,naux))      # nu_alpha,j
aux_e_init = np.random.random(naux) 

print(h_phys[:nphys,:nphys])

#Compute for supersystem
h_super_init = construct_h_super(aux_v_init, aux_e_init)
e_super_init, v_super_init = np.linalg.eigh(h_super_init)

# Construct all the X matrices here
X = [construct_X(e_super_init, v_super_init, mom)  for mom in range(n+1)]

# Construct all the T matrices here
T = [construct_T(mom)  for mom in range(n+1)]

param_init = np.zeros((naux+naux*nphys)) 
param_init[:naux] = aux_e_init
param_init[naux:] = np.reshape(aux_v_init, (nphys*naux))

# Initial error
error_init = mom_err(param_init, T)

print(X)

aux_e_final, aux_v_final = fitting(param_init)

h_super_final = construct_h_super(aux_v_final, aux_e_final)
e_super_final, v_super_final = np.linalg.eigh(h_super_final)

# Construct all the X matrices here
X = [construct_X(e_super_final, v_super_final, mom)  for mom in range(n+1)]

new_param = np.zeros((naux+naux*nphys)) 
new_param[:naux] = aux_e_final
new_param[naux:] = np.reshape(aux_v_final, (nphys*naux))

# Initial error
error_fin = mom_err(new_param, T)

print(X)

print('Init param', param_init)
print('Final param', new_param)
print('HL mom', T)
print('Init error', error_init)
print('Final error', error_fin)

[[0.96537487]]
[array([[1.]]), array([[0.96537487]]), array([[1.56558388]])]
Optimization terminated successfully.
         Current function value: 0.623380
         Iterations: 365
         Function evaluations: 644
[array([[1.]]), array([[0.96537487]]), array([[0.93194863]])]
Init param [0.20738711 0.26197366 0.01803879 0.72728429 0.22920552 0.45413226
 0.36245875 0.49344465]
Final param [ 2.96008713e-01  2.13129554e-01  4.41863214e-02  2.24823525e+00
 -5.20085119e-09 -1.32117665e-08  3.50117186e-09  6.23982155e-09]
HL mom [array([[1.]]), array([[0.82777553]]), array([[0.15448698]])]
Init error 2.01012801968397
Final error 0.6233801922459047
